<a href="https://colab.research.google.com/github/simonecaletti/arxiv-filter/blob/main/colab/yagmail_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install arxiv
!pip install python-dotenv
!pip install yagmail

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached arxiv-1.4.3-py3-none-any.whl (12 kB)
  Using cached feedparser-6.0.10-py3-none-any.whl (81 kB)
  Using cached sgmllib3k-1.0.0-py3-none-any.whl
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 KB 10.3 MB/s eta 0:00:00


# Arxiv-Filter

In [3]:
import os
import time
import re
from datetime import datetime 
from pytz import timezone
import requests
import arxiv
from dotenv import load_dotenv

load_dotenv()


False

In [33]:
class Query(object):
    def __init__(self, result):
        self.date = result.updated
        self.url = result.entry_id
        self.title = result.title
        self.authors = ', '.join([author.name for author in result.authors])
        self.abstract = result.summary
        # self.date_str = query['published']
        self.id = result.entry_id[result.entry_id.find("/v")+5:]
        self.categories = result.categories

    def is_recent(self):
        curr_time = datetime.now(timezone('GMT'))
        delta_time = curr_time - self.date
        assert delta_time.total_seconds() > 0
        return delta_time.days < 2

    def __hash__(self):
        return self.id

    def __str__(self):
        s = ''
        s += self.title + '\n'
        s += self.url + '\n'
        s += self.authors + '\n'
        s += ', '.join(self.categories) + '\n'
        s += self.date.ctime() + ' GMT \n'
        s += '\n' + self.abstract + '\n'
        return s
    
    def printQuery(self):
      print("============")
      print(self.date)
      print(self.url)
      print("Title:", self.title)
      print("Authors:", self.authors)
    
    def get_message_query(self):

      strdate = self.date.strftime("%d/%m/%Y")

      message = "===============\n"
      message += strdate + "\n"
      message += self.url + "\n"
      message += "Title: {}\n".format(self.title)
      message += "Authors: {}\n".format(self.authors)
      return message

In [51]:
class ArxivFilter(object):
  def __init__(self, categories, titles, authors, username):
    self._categories = categories
    self._titles = titles
    self._authors = authors
    self._username = username

  def _get_previously_sent_arxivs(self, _prev_arxivs=".prev_arxiv.txt"):
      if os.path.exists(_prev_arxivs):
          print("prev_arxiv.txt file does exist!")
          with open(_prev_arxivs, 'r') as f:
              return set(f.read().split('\n'))
      else:
          return set()

  def _save_previously_sent_arxivs(self, _new_queries, _prev_arxivs=".prev_arxiv.txt"):
      prev_arxivs = list(self._get_previously_sent_arxivs(_prev_arxivs))
      prev_arxivs += [q.id for q in _new_queries]
      prev_arxivs = list(set(prev_arxivs))
      with open(_prev_arxivs, 'w') as f:
          f.write('\n'.join(prev_arxivs))

  def _get_queries_from_last_day(self, max_results=100):
        queries1 = []
        queries2 = []

        category_query_string = "OR ".join([f"cat:{category} " for category in self._categories]).strip()
        title_query_string = "OR ".join([f"ti:\"{title}\" " for title in self._titles]).strip()
        auth_query_string = "OR ".join([f"au:\"{author}\" " for author in self._authors]).strip()
        query_string1 = f"({category_query_string}) AND ({title_query_string})"
        query_string2 = f"({category_query_string}) AND ({auth_query_string})"
        #print("Query string 1: ", query_string1)
        #print("Query string 2", query_string2)

        # get all queries in the categories in the last day filtered by title keywords
        search1 = arxiv.Search(query=query_string1, sort_by=arxiv.SortCriterion.SubmittedDate, max_results=max_results)
        new_queries1 = [Query(result) for result in search1.results()]
        queries1 += [q for q in new_queries1 if q.is_recent()]

        # get all queries in the categories in the last day filtered by author keywords
        search2 = arxiv.Search(query=query_string2, sort_by=arxiv.SortCriterion.SubmittedDate, max_results=max_results)
        new_queries2 = [Query(result) for result in search2.results()]
        queries2 += [q for q in new_queries2 if q.is_recent()]

        # merge the two queries and get rid of duplicates
        queries = queries1 + queries2
        queries_dict = {q.id: q for q in queries}
        unique_keys = set(queries_dict.keys())
        unique_queries = [queries_dict[k] for k in unique_keys]

        # sort from most recent to least
        sorted_queries = sorted(unique_queries, key=lambda q: (datetime.now(timezone('GMT')) - q.date).total_seconds())

        # filter if previously sent
        prev_arxivs = self._get_previously_sent_arxivs()
        prev_filtered_queries = [q for q in sorted_queries if q.id not in prev_arxivs]
        self._save_previously_sent_arxivs(prev_filtered_queries)
        
        return prev_filtered_queries 
  
  def _get_message(self, _queries):
    message = ""
    for q in _queries:
      message += q.get_message_query()
    return message
    
  def run(self):
    queries = self._get_queries_from_last_day()
    # for q in queries:
    #   q.printQuery()
    message = self._get_message(queries)
    return message


In [6]:
print("Before")
!ls .
!rm -rf *
print("After")
!ls .

Before
sample_data
After


In [7]:
from google.colab import files

categories = files.upload()
titles = files.upload()
authors = files.upload()

Saving categories.txt to categories.txt


Saving titles.txt to titles.txt


Saving authors.txt to authors.txt


In [8]:
with open('categories.txt', 'r') as f:
  categories = [line.strip() for line in f.read().split('\n') if len(line.strip()) > 0]

  print(categories)

['hep-ph', 'hep-th', 'quant-ph']


In [9]:
with open('titles.txt', 'r') as f:
  titles = [line.strip() for line in f.read().split('\n') if len(line.strip()) > 0]

print(titles)

['qcd', 'jets', 'jet', 'parton shower', 'quark', 'gluon', 'resummation', 'thrust', 'event shape', 'angularity', 'angularities', 'jet shape', 'strong coupling', 'Gradient', 'VQA', 'optimization', 'quantum annealing']


In [10]:
with open('authors.txt', 'r') as f:
  authors = [line.strip() for line in f.read().split('\n') if len(line.strip()) > 0]

print(authors)

['Simone Marzani', 'Gregory Soyez', 'Gavin Salam', 'Andrew J. Larkoski', 'Oleh Fedkevych', 'Jesse Thaler', 'Wouter Waalewijn', 'Gehrmann', 'Giovanni Stagnitto', 'Alexander Huss', 'Rhorry Gauld', 'Poncelet', 'Czakon', 'Mitov', 'Matteo Cacciari', 'Catani', 'Giulia Zanderighi', 'Andrea Banfi', 'Giovanni Ridolfi', 'Riccardo Torre']


In [55]:
af = ArxivFilter(categories=categories,
                 titles=titles,
                 authors=authors,
                 username="Simone")
emailtext = af.run()

In [54]:
!ls --all

.  ..  authors.txt  categories.txt  .config  titles.txt


In [47]:
!cat .prev_arxiv.txt

://arxiv.org/abs/2303.12691v1
://arxiv.org/abs/2303.12485v1
://arxiv.org/abs/2303.11200v2
://arxiv.org/abs/2303.12595v1
://arxiv.org/abs/2303.12063v1
://arxiv.org/abs/2303.12630v1
://arxiv.org/abs/2303.12520v1
://arxiv.org/abs/2303.12140v1
://arxiv.org/abs/2303.12475v1
://arxiv.org/abs/2303.12352v1
://arxiv.org/abs/2303.12601v1
://arxiv.org/abs/2303.12781v1

In [53]:
!rm -rf .prev_arxiv.txt

# Yagmail

In [57]:
print(emailtext)

22/03/2023
http://arxiv.org/abs/2303.12781v1
Title: Drell-Yan lepton-pair production: $q_T$ resummation at approximate N$^4$LL+N$^4$LO accuracy
Authors: Stefano Camarda, Leandro Cieri, Giancarlo Ferrera
22/03/2023
http://arxiv.org/abs/2303.12691v1
Title: Probing quark transverse momentum distributions in the Color Glass Condensate: quark-gluon dijets in Deep Inelastic Scattering at next-to-eikonal accuracy
Authors: Tolga Altinoluk, Nestor Armesto, Guillaume Beuf
22/03/2023
http://arxiv.org/abs/2303.11200v2
Title: Parent Hamiltonian reconstruction via inverse quantum annealing
Authors: Davide Rattacaso, Gianluca Passarelli, Angelo Russomanno, Procolo Lucignano, Giuseppe E. Santoro, Rosario Fazio
22/03/2023
http://arxiv.org/abs/2303.12630v1
Title: Exclusive heavy vector meson photoproduction on nuclei in NLO perturbative QCD
Authors: K. J. Eskola, V. Guzey, T. Löytäinen, H. Paukkunen, C. A. Flett
22/03/2023
http://arxiv.org/abs/2303.12601v1
Title: A real world test of Portfolio Optimizat

In [61]:
import yagmail

yag = yagmail.SMTP('iscrizioni.simone@gmail.com', 'FjQUD7W6wp7PemBa')
contents = emailtext
yag.send('simone@ge.infn.it', 'arxiv-filter', contents)


SMTPAuthenticationError: ignored